In [ ]:
# fix line range in the code
# add source code to the json file
import solc_json_parser
import os
import json
from solc_json_parser.combined_json_parser import CombinedJsonParser
metadata_path = '../../smartagent-dataset/AccessControl/SmartBugsWild/all_files.json'

import subprocess
with open(metadata_path, 'r') as f:
    metadata = json.load(f)
print (len(metadata))

def run_solgrep(file_path, function_name, contract_name):
    if "(" in function_name:
        function_name = function_name.split('(')[0]
    print ("get function", function_name)
    file_name = file_path.split('/')[-1]
    command = ['./solgrep_wrapper.sh', file_path, function_name, file_path.replace(file_name, f"solgrep_{function_name}.json")]

    result = subprocess.run(command,check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return
# __fallback__
def run_solgrep_fallback(file_path, contract_name):
    print ("get fallback", contract_name)
    file_name = file_path.split('/')[-1]
    command = ['./solgrep_wrapper.sh', file_path, '__fallback__', file_path.replace(file_name, f"solgrep_fallback.json")]
    result = subprocess.run(command,check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return

base_path  = os.path.dirname(os.path.abspath(metadata_path))
def fix_line_range_data_dict(file_to_insert):
    # file_to_insert = 'fp_dict.json'
    with open(file_to_insert, 'r') as f:
        data_dict = json.load(f)
    count = 0
    for key_,val_ in data_dict.items():
        if len(val_) < 1:
            continue
        for meta_k,meta_val in metadata.items():
            if key_ in meta_k:
                print ("match", key_, meta_val.get("file"))
                file_path = os.path.join(base_path,meta_val.get("file"))
                print (os.path.exists(file_path))
                for func_name in val_.keys():
                    if func_name == 'fallback':
                        run_solgrep_fallback(file_path, meta_val.get("contract")[0])
                    else:
                        run_solgrep (file_path, func_name, meta_val.get("contract")[0])
                    # source_code = get_function_code_from_file(file_path, func_name)
                    # if 'code' in val_[func_name].keys():
                    #     code_data = val_[func_name]['code']
                    #     raw = code_data['raw']
                    #     line_num = code_data['line_num']
                    #     new_code = fix_function_code_from_file(file_path, line_num,func_name )
                    #     print ("--- new code ---")
                    #     print (new_code)
                    #     print ("--- old code ---")
                    #     print (raw)
        # count = count + 1
        # if count > 5:
        #     break
        # break
    # print (data_dict)
    # write_file = file_to_insert.replace('.json', '_with_code_fixed.json')
    # with open(write_file, 'w') as f:
    #     json.dump(data_dict, f, indent=2)
fix_line_range_data_dict('fp_dict.json')

fix_line_range_data_dict('tp_dict.json')
#get_function_code_from_file()


In [ ]:
# update the code in the json file
# add source code to the json file
import solc_json_parser
import os
import json
from solc_json_parser.combined_json_parser import CombinedJsonParser
metadata_path = '../../smartagent-dataset/AccessControl/SmartBugsWild/all_files.json'


with open(metadata_path, 'r') as f:
    metadata = json.load(f)
print (len(metadata))

# "tag": "match-function: IUniswapV2Pair.symbol",
#       "info": "true",
#       "loc": [
#         243,
#         4,
#         243,
#         59
#       ]
def get_loc_from_solgrep(file_path, function_name, contract_name, ref_loc = None):
    if ref_loc is None:
        ref_loc = [0,0]
    if "(" in function_name:
        function_name = function_name.split('(')[0]
    file_name = file_path.split('/')[-1]
    solgrep_file = file_path.replace(file_name, f"solgrep_{function_name}.json")
    if not os.path.exists(solgrep_file):
        return None, None
    with open(solgrep_file, 'r') as f:
        data = json.load(f)
    if len(data) < 1:
        return None, None
    # min_loc = [-1,-1]
    # min_distance = 1000000
    max_range = 0
    max_range_loc = [-1,-1]
    for key_,val_ in data.items():
        if len(val_) < 1:
            return None, None
        for match_val in val_:
            # distance = match_val.get('loc')[0] - ref_loc[0]
            curr_range = match_val.get('loc')[2] - match_val.get('loc')[0]
            if (curr_range > 0) and (curr_range > max_range):
                max_range = curr_range
                max_range_loc = match_val.get('loc')
            if f"{contract_name}.{function_name}" in match_val.get('tag'):
                return match_val.get('loc')[0], match_val.get('loc')[2]
    if max_range_loc[0] >= 0:
        return max_range_loc[0], max_range_loc[2]
    return None, None
    # res = parser.function_by_name(,function_name)
    # print(res)
base_path  = os.path.dirname(os.path.abspath(metadata_path))
def fix_code_data_dict(file_to_insert):
    # file_to_insert = 'fp_dict.json'
    with open(file_to_insert, 'r') as f:
        data_dict = json.load(f)
    for key_,val_ in data_dict.items():
        if len(val_) < 1:
            continue
        for meta_k,meta_val in metadata.items():
            if key_ in meta_k:
                print ("match", key_, meta_val.get("file"))
                file_path = os.path.join(base_path,meta_val.get("file"))
                print (os.path.exists(file_path))
                # function_name = val_.get("location")
                # get_function_code_from_file(file_path, all_funcs[0])
                for func_name in val_.keys():
                    ref_loc = val_[func_name].get('code',{}).get('line_num')
                    start_line,end_line = get_loc_from_solgrep(file_path,
                                            func_name, meta_val.get("contract")[0])
                    if start_line is None or end_line is None:
                        continue
                    raw_file = open(file_path, 'r')
                    lines = raw_file.readlines()
                    new_code = '\n'.join(lines[start_line-1:end_line+1])
                    if 'code' in val_[func_name].keys():
                        code_data = val_[func_name]['code']
                        code_data['raw'] = new_code
                        code_data['line_num'] = [start_line, end_line]
                        val_[func_name]['code'] = code_data
                    else:
                        code_data = {
                            "raw": new_code,
                            "modifiers": [],
                            "line_num" : [start_line, end_line]
                        }
                        val_[func_name]['code'] = code_data
                    # if source_code:
                    #     # print ("found code", func_name, val_)
                    #     code_data = {
                    #         "raw": source_code.raw,
                    #         "modifiers": source_code.modifiers,
                    #         "line_num" : list(source_code.line_num)
                    #     }
                        # val_[func_name]['code'] = code_data
        # break
    # print (data_dict)
    write_file = file_to_insert
    with open(write_file, 'w') as f:
        json.dump(data_dict, f, indent=2)
# add_code_to_data_dict('fp_dict.json')
fix_code_data_dict('fp_dict_with_code_with_ir.json')
fix_code_data_dict('tp_dict_with_code_with_ir.json')
#get_function_code_from_file()